# Notebook for CS6378 Project
## Comparison of real-time filtering technique on FPGA vs. modern CPU
### Chris Tsongas cst130030@utdallas.edu
### Summary
The modules below can be run in sequential order to see the FPGA setup and running processes. The USE_FPGA parameter allows switching between running Sobel filter on the FPGA or within the Zynq CPU. 

To start, download the base overlay and instantiate the HDMI input and output.

In [1]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
import cv2, time, warnings
import numpy as np
warnings.filterwarnings('ignore')
import pynq
from pynq import Overlay
from pynq import allocate

overlay = BaseOverlay('base_w_sobel.bit')

sobel = overlay.sobel_0
dma = overlay.axi_dma_0

hdmi_in = overlay.video.hdmi_in
hdmi_out = overlay.video.hdmi_out

## Getting started

Initialize the Memory and Button Interrupt for buffering input video to the FPGA filter, Create the HDMI input and HDMI output streams as well. 

In [2]:
def dma_reset():
    dma.sendchannel.stop()
    dma.recvchannel.stop()
    dma.sendchannel.start()
    dma.recvchannel.start()
dma_reset()

def buttonInterrupt():
    return False if overlay.buttons[3].read() == 0 else True

hdmi_in.configure(PIXEL_GRAY)
hdmi_out.configure(hdmi_in.mode)
hdmi_in.cacheable_frames = False
hdmi_out.cacheable_frames = False
hdmi_in.start()
hdmi_out.start()

### Setup the shared/FPGA parameters
This function will initialize the FPGA Filter buffers and parameters

In [3]:
KERNEL_SIZE = 3
USE_FPGA = True
HORIZONTAL = hdmi_in.mode.width
VERTICAL = hdmi_in.mode.height
RESOLUTION = (VERTICAL, HORIZONTAL)
# To avoid the need for padding, set input and output to DMA as squares
FP_VBLANK = (max(RESOLUTION) - min(RESOLUTION)) // 2
BP_VBLANK = FP_VBLANK + min(RESOLUTION)
BUFFER_SHAPE = max(RESOLUTION)
# Static values used to initialize FPGA Sobel filter
FIRST_PIXEL_CMD = 0x10
LAST_PIXEL_CMD = 0x18
CONTROL_REG = 0x00
START_CMD = 0x81

input_buffer = allocate(shape=(BUFFER_SHAPE, BUFFER_SHAPE), dtype=np.uint8)
output_buffer = allocate(shape=(BUFFER_SHAPE, BUFFER_SHAPE), dtype=np.uint8)

### Function to call to run the Sobel filter on FPGA 

In [4]:
def run_sobel():
    #reset memory each frame
    dma_reset()
    # setup memory transfers
    dma.sendchannel.transfer(input_buffer)
    dma.recvchannel.transfer(output_buffer)
    # setup sobel HLS that is in-between memory interfaces
    sobel.write(FIRST_PIXEL_CMD, BUFFER_SHAPE)
    sobel.write(LAST_PIXEL_CMD, BUFFER_SHAPE)
    sobel.write(CONTROL_REG, START_CMD)
    # wait for receive and send to finish
    dma.sendchannel.wait()
    dma.recvchannel.wait()
    # output will be captured in output_buffer

### Full Pass-thru HDMI-IN $\rightarrow$ HDMI_OUT
While this provides for a fast way of passing video data through the pipeline there is no way to access or modify the frames. For that we a loop calling `readframe` and `writeframe`.

In [5]:
import time
numframes = 1
start = time.time()

while (buttonInterrupt() == False):
    fstime = time.time()
    f = hdmi_in.readframe()
    cv2.putText(f, "FPS: " + str(round(1/(time.time()-fstime),4)),(1,20),0,0.8,(255,255,255),1)
    hdmi_out.writeframe(f)
    numframes+=1

end = time.time()
print("Frames per second: " + str(numframes / (end - start)))

Frames per second: 58.76213415323896


Next we can start adding some filtering into the mix. 
The for loop will convert input to gray-scale then apply the filter based on parameters above.

In [6]:
result = np.ndarray(shape=RESOLUTION, dtype=np.uint8)

start = time.time()
numframes = 1
#while( numframes < 100 ):
while (buttonInterrupt() == False):
    fstime = time.time()
    ###############################################################
    # Read input from HDMI_IN
    ###############################################################
    inframe = hdmi_in.readframe()
    if ( numframes == 1 ):
        print("INPUT SHAPE : " + str(inframe.shape))
    if ( USE_FPGA ):
        # Only apply padding and send to memory when using FPGA hardware
        inframe = cv2.copyMakeBorder(inframe, FP_VBLANK, FP_VBLANK, 0, 0, cv2.BORDER_CONSTANT, value=[0,0,0])
        input_buffer[:,:] = inframe
    
    ###############################################################
    # Apply kernel
    ###############################################################
    outframe = hdmi_out.newframe()
    # sobel with custom FPGA hardware
    if ( USE_FPGA ):
        run_sobel()
    else:
        cv2.Laplacian(inframe, cv2.CV_8U,ksize=KERNEL_SIZE, dst=outframe)
    
    ###############################################################
    # Write output to HDMI_OUT
    ###############################################################
    
    if ( USE_FPGA ):
        # Remove the front and back porch of the memory to reformat into HDMI_IN size
        outframe = output_buffer[FP_VBLANK:BP_VBLANK, :]
    
    if ( numframes == 1 ):
        print("OUTPUT SHAPE : " + str(outframe.shape))
    # Commented out to avoid large FPS penalty
    
    cv2.putText(outframe, "FPS: " + str(round(1/(time.time()-fstime),4)),(1,20),0,0.8,(255,255,255),1)
    hdmi_out.writeframe(outframe)
    numframes+=1

end = time.time()
print("Frames per second: " + str(numframes / (end - start)))
        
        

INPUT SHAPE : (720, 1280)
OUTPUT SHAPE : (720, 1280)
Frames per second: 12.197324333434295


## Cleaning up

Finally you must always stop the interfaces when you are done with them. Otherwise bad things can happen when the bitstream is reprogrammed. You can also use the HDMI interfaces in a context manager to ensure that the cleanup is always performed.

In [ ]:
hdmi_out.close()
hdmi_in.close()

Re-running the plain read-write loop now shows 60 FPS

In [ ]:
numframes = 600
start = time.time()

for _ in range(numframes):
    f = hdmi_in.readframe()
    hdmi_out.writeframe(f)
    
end = time.time()
print("Frames per second:  " + str(numframes / (end - start)))

At the expense of much slower OpenCV performance

In [ ]:
numframes = 10
start = time.time()

for _ in range(numframes):
    inframe = hdmi_in.readframe()
    cv2.cvtColor(inframe,cv2.COLOR_BGR2GRAY,dst=grayscale)
    # inframe.freebuffer()
    cv2.Laplacian(grayscale, cv2.CV_8U, dst=result)

    outframe = hdmi_out.newframe()
    cv2.cvtColor(result, cv2.COLOR_GRAY2BGR,dst=outframe)
    hdmi_out.writeframe(outframe)
    
end = time.time()
print("Frames per second:  " + str(numframes / (end - start)))

In [ ]:
hdmi_out.close()
hdmi_in.close()

## Gray-scale
Using the new infrastructure we can delegate the color conversion to the hardware as well as only passing a single grayscale pixel to and from the processor.

First reconfigure the pipelines in grayscale mode and tie the two together to make sure everything is working correctly.

In [ ]:
base.download()

hdmi_in.configure(PIXEL_GRAY)
hdmi_out.configure(hdmi_in.mode)
hdmi_in.cacheable_frames = True
hdmi_out.cacheable_frames = True
hdmi_in.start()
hdmi_out.start()

hdmi_in.tie(hdmi_out)

Now we can rewrite the loop without the software colour conversion.

In [ ]:
start = time.time()

numframes = 30

for _ in range(numframes):
    inframe = hdmi_in.readframe()
    outframe = hdmi_out.newframe()
    cv2.Laplacian(inframe, cv2.CV_8U, dst=outframe)
    # inframe.freebuffer()
    hdmi_out.writeframe(outframe)
    
end = time.time()
print("Frames per second:  " + str(numframes / (end - start)))

In [ ]:
hdmi_out.close()
hdmi_in.close()

## Other modes

There are two other 24 bit modes that are useful for interacting with PIL. The first is regular RGB mode.

In [ ]:
base.download()

hdmi_in.configure(PIXEL_RGB)
hdmi_out.configure(hdmi_in.mode, PIXEL_RGB)

hdmi_in.start()
hdmi_out.start()

hdmi_in.tie(hdmi_out)

This is useful for easily creating and displaying frames with Pillow.

In [ ]:
import PIL.Image

frame = hdmi_in.readframe()
image = PIL.Image.fromarray(frame)
image

An alternative mode is YCbCr which is useful for some image processing algorithms or exporting JPEG files. Because we are not changing the number of bits per pixel we can update the colorspace of the input dynamically.

In [ ]:
hdmi_in.colorspace = COLOR_IN_YCBCR

It's probably worth updating the output colorspace as well to avoid the psychedelic  effects

In [ ]:
hdmi_out.colorspace = COLOR_OUT_YCBCR

Now we can use PIL to read in the frame and perform the conversion back for us.

In [ ]:
import PIL.Image

frame = hdmi_in.readframe()
image = PIL.Image.fromarray(frame, "YCbCr")
frame.freebuffer()
image.convert("RGB")

In [ ]:
hdmi_out.close()
hdmi_in.close()

## Next Steps

This notebook has only provided an overview of base overlay pipeline. One of the reasons for the changes was to make it easier to add hardware accelerated functions by supporting a wider range of pixel formats without software conversion and separating out the HDMI front end from the video DMA. Explore the code in pynq/lib/video.py for more details.